In [ ]:
from urllib.request import urlretrieve
urlretrieve('https://raw.githubusercontent.com/mneedham/neo4j-himym/master/data/import/sentences.csv', 'sentences.csv')

('sentences.csv', <http.client.HTTPMessage at 0x7f379200f190>)

In [ ]:
from collections import defaultdict
import csv

In [ ]:
#preprocessing data, read file and convert it into list containing content
episodes = defaultdict(list)
with open('sentences.csv', 'r') as sentences_file:
    reader = csv.reader(sentences_file, delimiter = ',')
    next(reader)
    for row in reader:
        episodes[row[1]].append(row[4])

for episode_id, text in episodes.items():
    episodes[episode_id] = "".join(text)

corpus = []
for ID, episode in sorted(episodes.items(), key = lambda t : int(t[0])):
    corpus.append(episode)

In [ ]:
episodes

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer = 'word', ngram_range=(1, 3), min_df = 0, stop_words = 'english')

#ngram_range: chung ta se tao ra nhung cum tu chua 2 den 3 tu cung voi nhung tu don co trong tap du lieu. Vi du 
#'Python is cool' se sinh ra dc cac tu va cum tu 'Python', 'is', 'cool', 'python is', 'python cool', 'is cool'.


In [ ]:
tfidf_matrix = tf.fit_transform(corpus)
feature_names = tf.get_feature_names()
len(feature_names)

498229

In [ ]:
feature_names[50:70]

['00 does sound',
 '00 don',
 '00 don buy',
 '00 dressed',
 '00 dressed blond',
 '00 drunkenly',
 '00 drunkenly slurred',
 '00 fair',
 '00 fair tonight',
 '00 fall',
 '00 fall foliage',
 '00 far',
 '00 far impossible',
 '00 fart',
 '00 fart sure',
 '00 friends',
 '00 friends singing',
 '00 getting',
 '00 getting guys',
 '00 god']

In [ ]:
#tu do chung ta co gan 500 000 cum tu, va bi the ma tran tf-idf cua chung ta se la 
#ma tran 208x499337 - moi mot hang bieu dien mot tap phi, moi mot cot bieu dien mot cum tu
tfidf_matrix

<208x498229 sparse matrix of type '<class 'numpy.float64'>'
	with 740365 stored elements in Compressed Sparse Row format>

In [ ]:
#lay ra tap cac tu va tf_idf cua no trong tap mot cua bo phim 
dense = tfidf_matrix.todense()
episode = dense[0].tolist()[0]
phrase_scores = [pair for pair in zip(range(0, len(episode)), episode) if pair[1] > 0]

len(phrase_scores)

4823

In [ ]:
phrase_scores

[(1875, 0.007401971584535902),
 (1894, 0.010873010595945407),
 (1896, 0.011713754336289437),
 (2454, 0.009991108452917341),
 (2921, 0.0049579960355101),
 (2941, 0.011713754336289437),
 (3148, 0.008376537832840345),
 (3150, 0.009435749678694228),
 (3325, 0.010873010595945407),
 (3326, 0.011713754336289437),
 (5064, 0.010873010595945407),
 (5065, 0.011713754336289437),
 (5066, 0.011713754336289437),
 (5624, 0.008178909392170244),
 (5641, 0.011713754336289437),
 (5642, 0.011713754336289437),
 (6374, 0.006098533175245135),
 (6455, 0.011713754336289437),
 (6456, 0.011713754336289437),
 (7328, 0.007468877231885245),
 (7357, 0.011713754336289437),
 (7358, 0.011713754336289437),
 (7697, 0.010276493419038258),
 (7699, 0.011713754336289437),
 (7842, 0.00883923250178708),
 (7843, 0.010873010595945407),
 (8805, 0.011440968207695674),
 (8825, 0.011713754336289437),
 (8826, 0.011713754336289437),
 (8858, 0.011713754336289437),
 (8859, 0.011713754336289437),
 (8891, 0.010276493419038258),
 (8894, 0.0

In [ ]:
#sap xep theo thu tu giam dan cua tf-idf 
sorted(phrase_scores, key=lambda t: t[1] * -1)[:5]

[(419182, 0.2625177493269755),
 (312566, 0.19571419072701732),
 (267513, 0.15551468983363487),
 (490404, 0.15227880637176266),
 (356607, 0.1304175242341549)]

In [ ]:
#Giá trị đầu tiên của mỗi tuple chính là index của cụm từ trong biến feature_names cho phép chúng ta lấy lại dạng văn bản của cụm từ.
feature_names[419408]

'ted 2030 mr'

In [ ]:
feature_names[312749]

'ones ted 64'

In [ ]:
sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
   print('{0: <20} {1}'.format(phrase, score))

ted                  0.2625177493269755
olives               0.19571419072701732
marshall             0.15551468983363487
yasmine              0.15227880637176266
robin                0.1304175242341549
barney               0.12441175186690791
lily                 0.12292497785945679
signal               0.1037932464656365
goanna               0.09813798750091524
scene                0.09534236041231685
cut                  0.09173366535740156
narrator             0.08646229819848741
flashback            0.07829592155397117
flashback date       0.07028252601773662
ranjit               0.06939276915589167
flashback date robin 0.05856877168144719
ted yasmine          0.05856877168144719
carl                 0.058210117288760355
eye patch            0.05436505297972703
lebanese             0.05436505297972703


In [ ]:
#tên nhân vật xuất hiện phần lớn ở đầu, đây là những giá trị không quan trọng vì nó ko mang ý nghĩa của văn bản 
# nhưng 'olives' quan trọng vì nó liên quan đến giả thuyết ovlies đc nhân vật đề cập trong tập đầu và ko xuất hiện ở các tập sau 
# 'yasmine' cũng là tên nhân vật nhưng chĩ xuất hiện ở những tập đuầ 
# hai tu này se là quan trọng trong những đoạn văn bản mà nó đc dê cập đến và ít quan trong so với những đoạn khác